# Linear Regression in Gluon
### Generating Data Sets

In [ ]:
!pip install mxnet

In [1]:
from mxnet import autograd, nd

num_inputs = 2
num_examples = 1000
true_w = nd.array([2, -3.4])
true_b = 4.2
features = nd.random.normal(scale=1, shape=(num_examples, num_inputs))
labels = nd.dot(features, true_w) + true_b
labels += nd.random.normal(scale=0.01, shape=labels.shape)

### Reading Data

In [2]:
from mxnet.gluon import data as gdata

batch_size = 10
# Combine the features and labels of the training data
dataset = gdata.ArrayDataset(features, labels)
# Randomly reading mini-batches
data_iter = gdata.DataLoader(dataset, batch_size, shuffle=True)

# Read a batch to see how it works
for X, y in data_iter:
    print(X, y)
    break


[[-0.45886332 -1.4475833 ]
 [-1.0442649   0.19647914]
 [-1.059102   -0.34620294]
 [-0.35445517 -1.3242855 ]
 [-0.542915    1.2974086 ]
 [ 0.14459428 -0.28338912]
 [ 0.6185597   0.21763243]
 [ 2.7692301  -0.15112336]
 [-0.06933286  1.1024144 ]
 [ 0.7247974  -0.8402941 ]]
<NDArray 10x2 @cpu(0)> 
[ 8.2049885   1.4411061   3.2603705   7.996832   -1.2931155   5.4458017
  4.698547   10.257168    0.30852094  8.513775  ]
<NDArray 10 @cpu(0)>


### Define the Model

In [3]:
from mxnet.gluon import nn
net = nn.Sequential()
net.add(nn.Dense(1))

### Initialize Model Parameters

- Initialize weight parameter by a normal distribution with a mean of 0 and standard deviation of 0.01.
- The bias parameter is initialized to zero by default.

In [4]:
from mxnet import init
net.initialize(init.Normal(sigma=0.01))

### Define the Loss Function

In [5]:
from mxnet.gluon import loss as gloss
loss = gloss.L2Loss()  # The squared loss is also known as the L2 norm loss

### Define the Optimization Algorithm

In [6]:
from mxnet import gluon
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.03})

### Training

In [16]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        with autograd.record():
            l = loss(net(X), y)
        l.backward()
        trainer.step(batch_size)
    l = loss(net(features), labels)
    print('epoch %d, loss: %f' % (epoch, l.mean().asnumpy()))

epoch 1, loss: 0.000050
epoch 2, loss: 0.000050
epoch 3, loss: 0.000050
epoch 4, loss: 0.000050
epoch 5, loss: 0.000050
epoch 6, loss: 0.000050
epoch 7, loss: 0.000050
epoch 8, loss: 0.000050
epoch 9, loss: 0.000050
epoch 10, loss: 0.000050


### Evaluate 

In [8]:
w = net[0].weight.data()
print('Error in estimating w', true_w.reshape(w.shape) - w)
b = net[0].bias.data()
print('Error in estimating b', true_b - b)
print(w)
print(b)

Error in estimating w 
[[ 0.00081837 -0.00019145]]
<NDArray 1x2 @cpu(0)>
Error in estimating b 
[0.00046682]
<NDArray 1 @cpu(0)>

[[ 1.9991816 -3.3998086]]
<NDArray 1x2 @cpu(0)>

[4.199533]
<NDArray 1 @cpu(0)>


#With Keras

In [9]:
import pandas as pd

import numpy as np
import keras

from matplotlib import pyplot as plt

from keras.layers import Input, Dense
from keras.models import Model

from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer

%matplotlib inline

Using TensorFlow backend.
/home/iserina/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/iserina/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/iserina/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/iserina/.local/lib/python3.6/site-packages/tensorflow/py

In [48]:
inputs = Input(shape=(2,))
preds = Dense(1,activation='linear')(inputs)
X_=features.asnumpy()
y_=labels.asnumpy()
model = Model(inputs=inputs,outputs=preds)
sgd=keras.optimizers.SGD()
model.compile(optimizer=sgd ,loss='mse',metrics=['mse'])
model.fit(X_,y_, batch_size=batch_size, epochs=num_epochs, shuffle=False)

Epoch 1/10
1000/1000 [==============================] - 0s 303us/step - loss: 10.8149 - mean_squared_error: 10.8149
Epoch 2/10
1000/1000 [==============================] - 0s 105us/step - loss: 0.1972 - mean_squared_error: 0.1972
Epoch 3/10
1000/1000 [==============================] - 0s 94us/step - loss: 0.0038 - mean_squared_error: 0.0038
Epoch 4/10
1000/1000 [==============================] - 0s 95us/step - loss: 1.7084e-04 - mean_squared_error: 1.7084e-04
Epoch 5/10
1000/1000 [==============================] - 0s 90us/step - loss: 1.0203e-04 - mean_squared_error: 1.0203e-04
Epoch 6/10
1000/1000 [==============================] - 0s 104us/step - loss: 1.0078e-04 - mean_squared_error: 1.0078e-04
Epoch 7/10
1000/1000 [==============================] - 0s 90us/step - loss: 1.0077e-04 - mean_squared_error: 1.0077e-04
Epoch 8/10
1000/1000 [==============================] - 0s 112us/step - loss: 1.0077e-04 - mean_squared_error: 1.0077e-04
Epoch 9/10
1000/1000 [============================

In [39]:
weights, biases = model.layers[1].get_weights()
print('Error in estimating w', true_w.asnumpy() - weights.reshape(true_w.shape))
print('Error in estimating b', true_b - biases)
print("weights=", weights)
print("biases=",biases)


Error in estimating w [-4.0531158e-05 -3.1280518e-04]
Error in estimating b [-0.00042057]
weights= [[ 2.0000405]
 [-3.3996873]]
biases= [4.2004204]


In [40]:
def squared_loss(y_hat, y):
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2

In [45]:
ynew = model.predict(X_)
train_l = squared_loss(ynew, y_)
train_t=
train_l.mean()

<tf.Tensor 'Mean_1:0' shape=(1000,) dtype=float32>